In [3]:
from bs4 import BeautifulSoup
from selenium import webdriver
import requests
import csv


In [2]:
!pip install geopy

In [1]:
import pandas as pd
import numpy as np

In [4]:
browser = webdriver.Chrome()

In [5]:
url = 'https://lenouvelliste.com/article/193660/138-institutions-denseignement-superieur-sont-reconnues-en-haiti'
browser.get(url)
soup = BeautifulSoup(browser.page_source,'html5lib')


In [6]:
print(soup.prettify())

<html>
 <head>
  <script src="https://www.googletagservices.com/activeview/js/current/osd.js?cb=%2Fr20100101">
  </script>
  <script async="" src="https://buttons-config.sharethis.com/js/5c8432dbcbb218001761da40.js">
  </script>
  <script async="" src="//www.google-analytics.com/analytics.js">
  </script>
  <script id="google_shimpl" src="https://pagead2.googlesyndication.com/pagead/js/r20200730/r20190131/show_ads_impl_fy2019.js">
  </script>
  <script async="" src="//pagead2.googlesyndication.com/pagead/js/adsbygoogle.js">
  </script>
  <script>
   (adsbygoogle = window.adsbygoogle || []).push({
          google_ad_client: "ca-pub-8045668094969779",
          enable_page_level_ads: true
     });
  </script>
  <meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
  <title>
   Le Nouvelliste -     138 institutions d’enseignement supérieur sont reconnues en Haïti
  </title>
  <meta content="rjCl278eiPAiY7MXB5eAho3n9Vd2mmHm3qXXMTEfu18" name="google-site-verification"/>
  <!

In [7]:
article =soup.find_all('article')

In [8]:
ps = article[0].find_all('p')

In [9]:
start_index = 0
end_index =0
for index in range(len(ps)):
    text = str(ps[index].get_text())
    if 'Institutions d’Enseignement Supérieur privées reconnues' in text:
        start_index = index
        
    if 'Institutions Publiques d’Enseignement Supérieur' in text:
        end_index = index
        break
        
start_index,end_index    

(10, 131)

In [10]:
shool,categorie,address,department,commune = [],[],[],[],[]
for p in ps[start_index+1:end_index]:
    shool.append(p.get_text().strip())
    categorie.append('Private')
    address.append('')
    department.append('')
    commune.append('')
for p in ps[end_index+1:len(ps)]:
    shool.append(p.get_text().strip())
    categorie.append('Public')
    address.append('')
    department.append('')
    commune.append('')
df = pd.DataFrame(dict(shool=shool,categorie=categorie,address=address,department=department,commune=commune))
    

In [11]:
#df.to_csv('universities.csv')

In [2]:
df_university = pd.read_csv("university.csv", encoding = "latin8")

In [3]:
df_university.head()

,Universités/Instituts Supérieurs,Catégories,Adresses,Départments,Villes
0,Université Adventiste dHaïti (UNAH),Privé,"Route de la Mairie de Carrefour, Diquini 63, C...",Ouest,Carrefour
1,Université Américaine des Cayes (UAC) ci-devan...,Privé,"Route Nationale 2, Arrondissement des Cayes",Sud,Cayes
2,Université Américaine des Sciences Modernes d...,Privé,"Delmas 65, Rue Neptune, Port-au-Prince",Ouest,Delmas
3,Université Autonome de Port-au-Prince (UNAP),Privé,"Santo 25, Rue Robert #05, Croix-des-Bouquets, ...",Ouest,Croix-des-Bouquets
4,Université Autonome Charlemagne Péralte (UNACP),Privé,"10 Impasse Ambroise, Port-au-Prince",Ouest,Port-au-Prince


In [12]:
from geopy.geocoders import Nominatim
Names,Latitudes, Longitudes, Locations =[], [], [], []
for name,adr in zip(df_university['Universités/Instituts Supérieurs'],df_university.Adresses):
    geolocator = Nominatim(user_agent="university")
    location = geolocator.geocode(adr)
    print(name)
    if location:
        Latitudes.append(location.latitude)
        Longitudes.append(location.longitude)
        Locations.append((location.latitude, location.longitude))
        Names.append(name)
    


Université Adventiste dHaïti (UNAH)
Université Américaine des Cayes (UAC) ci-devant American University of the Caribean.
Université Américaine des Sciences Modernes dHaïti (UNASMOH)
Université Autonome de Port-au-Prince (UNAP)
Université Autonome Charlemagne Péralte (UNACP)
Université Bellevue (UNIBEL)
Université Caraïbe (UC)
Université Chrétienne du Nord dHaïti (UCNH)
Université Chrétienne de la Communauté de Caïman (UCCC)
Université de la Concorde / Concord University, ci-devant CCHEC-CUMES
Université de la Fondation Aristide (UFA)
Université de Port-au-Prince (UP)
Université des Sciences Administratives et de la Santé (UNISAS)
Université dEtudes Internationales (UDEI)
Université Evangélique dHaïti (UNEH)
Université Episcopale dHaïti (UNEPH)
Université INUKA
Université GOC
Université Intégrée de la Caraïbe (UNICA)
Université Jean-Price Mars (UJPM)
Université Lumière (UL)
Universite Libre dHaïti (ULH)
Université Métropole dHaïti (UMH)
Université Notre Dame dHaïti (UNDH)
Univ

In [13]:
val =dict(Names=Names,Latitudes=Latitudes,Longitudes=Longitudes,Locations=Locations)

In [16]:
df_123 =pd.DataFrame(val)

In [17]:
df_123

,Names,Latitudes,Longitudes,Locations
0,Université Américaine des Cayes (UAC) ci-devan...,18.213899,-73.749760,"(18.2138993, -73.7497597)"
1,Université Autonome Charlemagne Péralte (UNACP),18.541714,-72.288799,"(18.5417136, -72.2887988)"
2,Université Bellevue (UNIBEL),18.557261,-72.304711,"(18.5572606, -72.3047109)"
3,Université Caraïbe (UC),18.551846,-72.292007,"(18.5518464, -72.2920071)"
4,Université Chrétienne du Nord dHaïti (UCNH),19.749755,-72.207547,"(19.7497554, -72.2075468)"
...,...,...,...,...
56,Faculté de Médecine et de Pharmacie (FMP),18.538411,-72.339242,"(18.5384108, -72.3392416)"
57,Institut National dAdministration de Gestion ...,18.540736,-72.332746,"(18.5407356, -72.3327464)"
58,Ecole de Droit et des Sciences Economiques des...,18.207148,-73.758538,"(18.2071475, -73.7585384)"
59,Ecole de Droit de Fort-liberté,19.656639,-71.833582,"(19.6566394, -71.8335819)"


In [20]:
df_university = df_university.join(df_123,how='left',sort=False)

In [21]:
df_university

,Universités/Instituts Supérieurs,Catégories,Adresses,Départments,Villes,Names,Latitudes,Longitudes,Locations
0,Université Adventiste dHaïti (UNAH),Privé,"Route de la Mairie de Carrefour, Diquini 63, C...",Ouest,Carrefour,Université Américaine des Cayes (UAC) ci-devan...,18.213899,-73.749760,"(18.2138993, -73.7497597)"
1,Université Américaine des Cayes (UAC) ci-devan...,Privé,"Route Nationale 2, Arrondissement des Cayes",Sud,Cayes,Université Autonome Charlemagne Péralte (UNACP),18.541714,-72.288799,"(18.5417136, -72.2887988)"
2,Université Américaine des Sciences Modernes d...,Privé,"Delmas 65, Rue Neptune, Port-au-Prince",Ouest,Delmas,Université Bellevue (UNIBEL),18.557261,-72.304711,"(18.5572606, -72.3047109)"
3,Université Autonome de Port-au-Prince (UNAP),Privé,"Santo 25, Rue Robert #05, Croix-des-Bouquets, ...",Ouest,Croix-des-Bouquets,Université Caraïbe (UC),18.551846,-72.292007,"(18.5518464, -72.2920071)"
4,Université Autonome Charlemagne Péralte (UNACP),Privé,"10 Impasse Ambroise, Port-au-Prince",Ouest,Port-au-Prince,Université Chrétienne du Nord dHaïti (UCNH),19.749755,-72.207547,"(19.7497554, -72.2075468)"
...,...,...,...,...,...,...,...,...,...
118,Ecole de Droit et des Sciences Economiques de ...,Public,"33, rue Bénito Sylvain, Port-de-Paix",Nord-Ouest,Port-de-Paix,NaN,NaN,NaN,NaN
119,Ecole Nationale des Infirmières de Port-au-Pri...,Public,89 Rue Oswald Durant Port-au-Prince Haïti,Ouest,Port-au-prince,NaN,NaN,NaN,NaN
120,Ecole Nationale des Infirmières des Cayes,Public,"Rue Monseigneur Morice, Cayes, Haïti",Sud,Cayes,NaN,NaN,NaN,NaN
121,Ecole Nationale des Infirmières de Jérémie,Public,"38, Rue Brouette en face de lévêché de Jérémi...",Grande-Anse,Jeremie,NaN,NaN,NaN,NaN


In [26]:
df_university.tail(40)

,Universités/Instituts Supérieurs,Catégories,Adresses,Départments,Villes,Names,Latitudes,Longitudes,Locations
83,Académie de Formation et de Perfectionnement d...,Privé,"12, Rue Pacot, (entrée Rue 4)",Ouest,Port-au-Prince,NaN,NaN,NaN,NaN
84,Burnett International University (BIU),Privé,"13 Blvd 15 Octobre, en Face Tabarre 65",Ouest,Tabarre,NaN,NaN,NaN,NaN
85,Grand Séminaire Notre Dame (GSND),Privé,"110, rue Lafleur Ducheine",Ouest,Port-au-Prince,NaN,NaN,NaN,NaN
86,Haïtian Education and Leadership Program (HELP),Privé,"# 4, Rue 5, Pacot, Port-au-Prince",Ouest,Port-au-Prince,NaN,NaN,NaN,NaN
87,Séminaire de Théologie Évangélique de Port-au-...,Privé,"261 Route des Dalles,Port-au-Prince",Ouest,Port-au-Prince,NaN,NaN,NaN,NaN
88,Spring Hill University (SHU),Privé,"46, RUE BERGEAU, DELMAS 18",Ouest,Delmas,NaN,NaN,NaN,NaN
89,Université Publique du Nord au Cap-Haïtien (UP...,Public,"Balan Habitation Baudin, Cap-Haïtien",Nord,Cap-Haïtien,NaN,NaN,NaN,NaN
90,Université Publique de lArtibonite aux Gonaïv...,Public,"45 rue Chrysostome Humbert, Gonaïves",Artibonite,Gonaïves,NaN,NaN,NaN,NaN
91,Université Publique du Sud aux Cayes (UPSAC),Public,"Campus de Laborde, Les Cayes BP 131, Les Cayes",Sud,Cayes,NaN,NaN,NaN,NaN
92,Université Publique du Sud-Est à Jacmel (UPSEJ),Public,"Route de Meyer, Jacmel, Haïti",Sud-Est,Jacmel,NaN,NaN,NaN,NaN


In [27]:
df_university.to_csv('df_university.csv')